In [11]:
import os
import glob
import sys

import numpy as np
import pandas as pd

from progressbar import ProgressBar

In [12]:
FEATURES_RP_EXTRACTED_PATH = "E:/Data/MIR/EU_SOUND_CHANNEL/features"

FEATURES_RP_DST_PATH       = "E:/Features/MIR/EuropeanaSounds_raw/audio_features/rp_extract"

# Process RP-Features

RP_extract results are all in one directory. Features have been extracted on a directory level. Resulting feature files carry the name of the processed directory and contain the corresponding features of the file extension. Within the feature files, each line starts with the filename of the processed audio file, followed by the feature vector

In [13]:
pbar = ProgressBar()

# process all feature files
for feature_file_path in pbar(glob.glob(os.path.join(FEATURES_RP_EXTRACTED_PATH, "*.*"))):
    
    feature_file_path = feature_file_path.replace("\\", "/")
    
    # define destination paths
    feature_file_name = feature_file_path.split("/")[-1]
    collection_name   = feature_file_name.split(".")[0]
    feature_name      = feature_file_name.split(".")[1]
    feature_dst_dir   = "%s/%s/%s" % (FEATURES_RP_DST_PATH, feature_name, collection_name)
    
    # create dir
    if not os.path.exists(feature_dst_dir):
        os.makedirs(feature_dst_dir)
    
    print feature_file_path, collection_name, feature_name
    
    try:
        
        feature_data = pd.read_csv(feature_file_path, sep=",", header=None, skip_blank_lines=True)
        
        for row in feature_data.iterrows():
            
            src_filename   = str(row[1][0]).replace(".mp3", "")
            feature_vector = row[1][1:]
            
            feature_file_dst_path = "%s\%s.csv" % (feature_dst_dir, src_filename)
            
            pd.DataFrame(feature_vector.values.reshape(1,feature_vector.values.shape[0])).to_csv(feature_file_dst_path, 
                                                                                                 header=None, 
                                                                                                 index=None, 
                                                                                                 sep=";", 
                                                                                                 float_format='%.6f')
        
    except pd.io.common.EmptyDataError as ede:
        print "*** expected error:", ede
    except Exception as e:
        print "*** unexpected error:", e

E:/Data/MIR/EU_SOUND_CHANNEL/features/09334.mvd 09334 mvd
*** expected error: No columns to parse from file
E:/Data/MIR/EU_SOUND_CHANNEL/features/09334.rh 09334 rh
*** expected error: No columns to parse from file
E:/Data/MIR/EU_SOUND_CHANNEL/features/09334.rp 09334 rp
*** expected error: No columns to parse from file
E:/Data/MIR/EU_SOUND_CHANNEL/features/09334.ssd 09334 ssd
*** expected error: No columns to parse from file
E:/Data/MIR/EU_SOUND_CHANNEL/features/09334.trh 09334 trh
*** expected error: No columns to parse from file
E:/Data/MIR/EU_SOUND_CHANNEL/features/09334.tssd 09334 tssd
*** expected error: No columns to parse from file
E:/Data/MIR/EU_SOUND_CHANNEL/features/2023601.mvd 2023601 mvd


  7% ( 6 of 78) |########                                                                                                 | Elapsed Time: 0:00:00 ETA:  0:00:00

KeyboardInterrupt: 

# Librosa Features

Librosa features have been filename encoded <collection-name>-<filename>.featurename

re-structure and rename to <collection-name>/<filename>.featurename

In [2]:
import shutil

In [3]:
LIBROSA_FEATURES_ROOT = "E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features"
LIBROSA_FEATURES_DST  = "E:/Features/MIR/EuropeanaSounds_raw/audio_features/librosa_raw"

In [4]:
# directories under root group features
for feature_dir in glob.glob(os.path.join(LIBROSA_FEATURES_ROOT, "*")):
    
    feature_dir = feature_dir.replace("\\", "/")
    
    print feature_dir
    
    feature_name = feature_dir.split("/")[-1]
    
    feature_dst_dir = "%s/%s" % (LIBROSA_FEATURES_DST, feature_name)
    
    for csv_file_path in glob.glob(os.path.join(feature_dir, "*.csv")):
        
        csv_file_path = csv_file_path.replace("\\", "/")
        
        csv_filename = csv_file_path.split("/")[-1]
        
        tmp = csv_filename.split("-")
        collection_name = tmp[0]
        filename = tmp[1]
        
        feature_file_dst_dir = "%s/%s" % (feature_dst_dir, collection_name)
        
        if not os.path.exists(feature_file_dst_dir):
            os.makedirs(feature_file_dst_dir)
        
        #print "move:", csv_file_path, "%s/%s" % (feature_file_dst_dir, filename)
        
        shutil.move(csv_file_path, "%s/%s" % (feature_file_dst_dir, filename))
        
        
        
        

E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/chroma
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/mfcc
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/rmse
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/spectral_bandwidth
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/spectral_centroid
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/spectral_contrast
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/spectral_rolloff
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/tonnetz
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features/zero_crossing_rate


# Build Intersection of Available data-items

In [14]:
path_metadata = "E:/Features/MIR/EuropeanaSounds_raw/metadata_rdf" # = reference


data_collections = [
    ["audio", "E:/Data/MIR/EU_SOUND_CHANNEL/audio", ".mp3"],
    ["librosa", "E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/chroma", ".csv"],
    ["rp_extract", "E:/Features/MIR/EuropeanaSounds/audio_features/rp_extract/mvd", ".csv"],
    ["librosa_npz", "E:/Features/MIR/EuropeanaSounds_raw/audio_features/npz_features", ".npz"]
]

** Metadata **

In [16]:
# read csv-data (separated by semicolons)
metadata = pd.read_csv("E:/Features/MIR/EuropeanaSounds/metadata/metadata.csv", sep=";")

# convert nan-values to empty strings
metadata = metadata.fillna("")

dataset = pd.DataFrame(metadata.id)

dataset["metadata"] = True

print metadata.shape

dataset = dataset.set_index("id")


for collection in data_collections:
    
    print collection

    data_files = []

    for collection_dir in glob.glob(os.path.join(collection[1], "*")):
        if os.path.isdir(collection_dir):
            for data_file in glob.glob(os.path.join(collection_dir, "*%s" % collection[2])):
                data_file = data_file.replace("\\", "/").replace(collection[1], "").replace(collection[2], "")
                data_files.append(data_file)

    collection_dataset = pd.DataFrame(data_files)
   
    
    collection_dataset = collection_dataset.set_index(0)
    del collection_dataset.index.name
    collection_dataset[collection[0]] = True
    
    dataset = dataset.join(collection_dataset, how="left").fillna(False)

dataset.head()

(32592, 20)
['audio', 'E:/Data/MIR/EU_SOUND_CHANNEL/audio', '.mp3']
['librosa', 'E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/chroma', '.csv']
['rp_extract', 'E:/Features/MIR/EuropeanaSounds/audio_features/rp_extract/mvd', '.csv']
['librosa_npz', 'E:/Features/MIR/EuropeanaSounds_raw/audio_features/npz_features', '.npz']


,metadata,audio,librosa,rp_extract,librosa_npz
id,,,,,
/09301/02DABC7F5850EABCA553BCEEEEAE2B6543A1CA67,True,False,False,False,False
/09301/03DE04818BECB920A21EA63EA9D5FD1C49CF74A0,True,False,False,False,False
/09301/0701AE796F7B504E67B8CEB031C1061E113BE3F4,True,False,False,False,False
/09301/0A3E8D891310E10CEC0B6D7790ADE1470E6541B8,True,False,False,False,False
/09301/0BF6DFA79787E76872BA33061C84A79588318953,True,False,False,False,False


In [17]:
print "metdata:",     dataset.shape
print "audio:",       dataset[dataset.audio].shape
print "librosa:",     dataset[dataset.librosa].shape
print "rp_extract",   dataset[dataset.rp_extract].shape
print "intersection", dataset[(dataset.audio) & (dataset.librosa) & dataset.rp_extract].shape

metdata: (32592, 5)
audio: (18820, 5)
librosa: (15193, 5)
rp_extract (12587, 5)
intersection (12587, 5)


In [44]:
pd.DataFrame(dataset[(dataset.audio) & (dataset.librosa) & dataset.rp_extract].index.values).to_csv("E:/Features/MIR/EuropeanaSounds/metadata/intersection.csv", 
                                                                                                    header=None,
                                                                                                   index=None)

In [33]:
dataset.index = dataset.index.astype(str)

In [32]:
metadata.index = metadata.index.astype(str)

# Aggregate Metadata

In [84]:
data_files = []

for collection_dir in glob.glob(os.path.join(path_metadata, "*")):
    if os.path.isdir(collection_dir):
        for data_file in glob.glob(os.path.join(collection_dir, "*.rdf")):
            data_file = data_file.replace("\\", "/")
            data_files.append(data_file)

In [93]:
tags = [
"dc:relation",
"dcterms:spatial",
"dc:language",
"dc:description",
"dc:identifier",
"dc:type",
"dc:title",
"dc:subject",
"dc:type",
"dc:description",
"dc:publisher",
"dc:creator",
"dc:contributor",
"edm:country",
"edm:provider",
"edm:dataProvider",
"edm:year",
"dcterms:medium",
"dc:date",
"dc:format",
"dcterms:created"
]

pbar = ProgressBar()

data = []

for df in pbar(data_files):

    data_item = {}
    
    data_item["id"] = df.replace("\\", "/").replace(path_metadata, "").replace(".rdf", "")
    
    for t in tags:
        data_item[t.split(":")[1]] = ""
    
    
    
    with open(df, 'r') as rdf:
        for line in rdf.readlines():

            if any(tag in line for tag in tags):
                
                try:
                    line = line.strip()
                    tmp = line.split(">")
                    tag = tmp[0].split(":")[-1]
                    content = tmp[1].split("<")[0]

                    if len(data_item[tag]) > 0:
                        data_item[tag] += ", %s" % content
                    else:
                        data_item[tag] = content
                except:
                    pass
                    
    data.append(data_item)
    
data = pd.DataFrame(data)
data.to_csv("E:/Features/MIR/EuropeanaSounds/metadata/metadata.csv", sep=";", index=None)

100% (32592 of 32592) |###################################################################################################| Elapsed Time: 0:06:39 Time: 0:06:39


In [97]:
["a", "b"].remove("b")

In [101]:
cols = data.columns.tolist()
cols.remove("id")
d = data[["id"] + cols]
d = d.fillna("")
d.to_csv("E:/Features/MIR/EuropeanaSounds/metadata/metadata.csv", sep=";", index=None)

# join separated feature vectors into a single file

In [39]:
FEATURES_PATH = "E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/"

In [40]:
# directories under root group features
for feature_dir in glob.glob(os.path.join(FEATURES_PATH, "*")):
    
    feature_dir = feature_dir.replace("\\", "/")
    
    if not os.path.isdir(feature_dir):
        continue
    
    track_ids = []
    data = []
    
    feature_name = feature_dir.split("/")[-1]
    
    print (feature_name, feature_dir)
    
    for collection_dir in glob.glob(os.path.join(feature_dir, "*")):
        
        collection_dir = collection_dir.replace("\\","/")
        
        if os.path.exists("%s/%s.csv" % (FEATURES_PATH, feature_name)):
            continue
        
        if os.path.isdir(collection_dir):
            
            pbar = ProgressBar()
            
            print collection_dir
            
            for feature_vector_file in pbar(glob.glob(os.path.join(collection_dir, "*.csv"))):
                
                feature_vector_file = feature_vector_file.replace("\\", "/")
                
                track_id = feature_vector_file.replace("%s" % (feature_dir), "").replace(".csv", "")
                
                vector = pd.read_csv(feature_vector_file, sep=";", header=None)
                
                
                track_ids.append(track_id)
                data.append(vector.values.tolist())
        
    data = np.asarray(data)
    if len(data.shape) == 3:
        data = data.reshape((data.shape[0], data.shape[2]))

    output = pd.DataFrame(data)
    output.insert(0, "id", track_ids)
    output = output.set_index("id")
    
    output.to_csv("%s/%s.csv" % (FEATURES_PATH, feature_name), sep=";", header=None)


('chroma', 'E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/chroma')
E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/chroma/2023601


100% (20785 of 20785) |###################################################################################################| Elapsed Time: 0:03:10 Time: 0:03:10
 81% ( 9 of 11) |#####################################################################################                    | Elapsed Time: 0:00:00 ETA:  0:00:00

E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/chroma/2048401


100% (11 of 11) |#########################################################################################################| Elapsed Time: 0:00:00 Time: 0:00:00
  4% ( 4 of 86) |####                                                                                                     | Elapsed Time: 0:00:00 ETA:  0:00:00

E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/chroma/2051921


...

E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/zero_crossing_rate/2059210


100% (2767 of 2767) |#####################################################################################################| Elapsed Time: 0:00:02 Time: 0:00:02


E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/zero_crossing_rate/2059213


100% (1077 of 1077) |#####################################################################################################| Elapsed Time: 0:00:01 Time: 0:00:01
100% (2 of 2) |###########################################################################################################| Elapsed Time: 0:00:00 Time: 0:00:00


E:/Features/MIR/EuropeanaSounds_raw/audio_features/csv_features_aggregated_new/zero_crossing_rate/9200257
